In [437]:
import numpy as np
from scipy.optimize import newton
import pandas as pd

In [438]:
typical_unit = pd.read_csv('typical_unit.csv')
typical_unit

,Average height,Footprint,Roof area,Floor area ratio,Building area,Number,OFF,RES,IND,OTH,Cluster
0,10.909091,0.197174,202239.20120,1.998721,2.050100e+06,1177.0,23.294289,39.909588,5.759907,31.036217,0.0
1,7.051793,0.016650,93180.15214,0.117903,6.598134e+05,502.0,30.172465,23.400985,0.450657,45.975893,1.0
2,13.388060,0.367417,19477.53425,5.072184,2.688870e+05,67.0,9.443528,0.000000,90.556472,0.000000,2.0
3,26.500000,0.333014,36428.15907,9.076837,9.929075e+05,36.0,88.131410,0.000000,11.868590,0.000000,3.0
4,7.558442,0.213726,19056.18943,1.581319,1.409930e+05,77.0,82.764523,0.237438,0.000000,16.998039,4.0
5,12.805556,0.293953,46395.28944,3.726849,5.882171e+05,432.0,3.641184,91.713401,3.984771,0.660644,5.0


#### 逐时数据准备

In [439]:
# 六个典型小区的总光伏和能耗
typical_EN = pd.read_csv('combined_sumcon.csv')
typical_PV = pd.read_csv('typical_PV.csv')

columns_to_process = ['0', '1', '5', '6', '8', '22']

# 创建一个字典，用于存储重组后的 DataFrame
reorganized_data = {}

# 遍历需要操作的列名
for col_name in columns_to_process:
    # 提取对应列的数据
    data_PV = typical_PV[col_name]/1.334886976
    data_EN = typical_EN[col_name]
    
    # 创建新的 DataFrame
    df = pd.DataFrame({'PV': data_PV, 'EN': data_EN})
    
    # 将新的 DataFrame 放入字典中，以列名作为键
    reorganized_data[f'SC{col_name}'] = df

# 通过字典访问生成的 DataFrames
SC0 = reorganized_data['SC0']
SC1 = reorganized_data['SC1']
SC5 = reorganized_data['SC5']
SC6 = reorganized_data['SC6']
SC8 = reorganized_data['SC8']
SC22 = reorganized_data['SC22']

In [442]:
# 判断光伏自用、电网供电和光伏上网

def classification(SCn):
    SCn['PVcharge'] = SCn.apply(lambda x: min(x['EN'], x['PV']), axis=1)
    SCn['gridcharge'] = SCn['EN'] - SCn['PVcharge']
    SCn['PV2G'] = SCn['PV']-SCn['PVcharge']
    return SCn

classified_SC5 = classification(SC5)

# classified_SC0.to_csv('classified_SC0.csv', index=False)

In [443]:
classified_SC5

,PV,EN,PVcharge,gridcharge,PV2G
0,0.0,5212.377598,0.0,5212.377598,0.0
1,0.0,5222.233432,0.0,5222.233432,0.0
2,0.0,5521.991365,0.0,5521.991365,0.0
3,0.0,5875.219935,0.0,5875.219935,0.0
4,0.0,6297.589031,0.0,6297.589031,0.0
...,...,...,...,...,...
8755,0.0,8950.783311,0.0,8950.783311,0.0
8756,0.0,8724.898358,0.0,8724.898358,0.0
8757,0.0,7705.160904,0.0,7705.160904,0.0
8758,0.0,6491.797565,0.0,6491.797565,0.0


In [444]:
classified_SC5['PV'].sum()

26996319.92045037

#### 能量

In [445]:
# 求总发电量

column_sums = typical_PV.sum()/1.334886976 # 数据补齐后存在参数
print("Sum of each column in typical_PV:")
print(column_sums)


Sum of each column in typical_PV:
0     2.576520e+06
1     6.244251e+06
5     2.699632e+07
6     2.482435e+06
8     1.232355e+07
22    4.304789e+06
dtype: float64


In [446]:
# 求总能耗

column_sums = typical_EN.sum()
print("Sum of each column in typical_EN:")
print(column_sums)

Sum of each column in typical_EN:
0     5.789079e+06
1     4.875568e+07
5     1.166639e+08
6     9.911961e+06
8     3.573033e+07
22    4.166429e+07
dtype: float64


In [447]:
# 求自消耗率

pvcharge_sum = classified_SC5['PVcharge'].sum()
pv_sum = classified_SC5['PV'].sum()

# 计算 PVcharge 列和 PV 列的比值
Self_consumption5 = pvcharge_sum / pv_sum
print("Self_consumption:", Self_consumption5)

Self_consumption: 0.9818341730713696


光伏自消耗率：指光伏发电系统中，自身消耗的电力占总发电量的比例。光伏自消耗率越低，说明光伏发电系统的能源利用率越高，系统运行效率越好。

In [448]:
# 求自满足率

pvcharge_sum = classified_SC5['PVcharge'].sum()
en_sum = classified_SC5['EN'].sum()

# 计算 PVcharge 列和 PV 列的比值
Self_consumption5 = pvcharge_sum / en_sum
print("Self_sufficiency:", Self_consumption5)

Self_sufficiency: 0.22719892721098509


指光伏发电系统中，自身能够满足的电力需求占总需求量的比例。光伏自满足率越高，说明光伏发电系统对自身电力需求的满足程度越好，从而减少了对传统能源的依赖，提高了经济性和环保性。

#### 环境

In [449]:
# 运行阶段碳排放
# 假设已经获得了光伏发电和能耗数据

total_energy_consumption = classified_SC5['EN'].sum()

emission_factor_electricity = 0.3748 # kgCO2/kWh

# 计算全年运行阶段碳排放量
total_emission = total_energy_consumption * emission_factor_electricity


print("Total energy consumption:", total_energy_consumption)
print("Total carbon emission:", total_emission)

Total energy consumption: 116663884.68661734
Total carbon emission: 43725623.98054418


#### 经济

In [450]:
def priceGrid(lst):
    step=[0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 0]
    bill=np.zeros([len(lst)])
    for i in range(len(lst)):
        h=i%24
        if step[h]==0:
            bill[i]=lst[i]*0.1911
        if step[h]==1:
            bill[i]=lst[i]*0.6603
        if step[h]==2:
            bill[i]=lst[i]*0.8949
    return (bill)

In [476]:
# 自发自用余电上网

# 自发自用省的
hour_price_grid = priceGrid(classified_SC1['PVcharge'])
tot_price_grid = np.sum(hour_price_grid)


# 光伏上网赚的
price_PV = 0.4530 * classified_SC1['PV2G'].sum()

# 政府补贴的
gov_price = 0.3 * classified_SC1['PV2G'].sum()

benefit = tot_price_grid + price_PV + gov_price


benefit / 10000  # 用光伏后每年的收益，单位：万元

473.3456747222449

#### 此处调用面积的时候需要注意对应顺序，光伏能耗数据是按照顺序的，但unit不是

In [475]:
typical_unit.loc[4, 'Roof area'] * 1000 / 10000  # 初始投资

1905.618943

#### 计算IRR

In [485]:
import numpy_financial as npf
from scipy.optimize import fsolve

# 初始投资
initial_investment = -typical_unit.loc[5, 'Roof area'] * 1000/10000  # 负数表示投资支出，单位万元

# 每年净收入，包括初始投资后的收入
net_inflows = [benefit / 10000] * 25  # 创建包含25个年收入的列表

# 计算IRR函数
def irr_func(rate):
    return npf.npv(rate, [initial_investment] + net_inflows)

# 使用fsolve函数计算IRR
irr = fsolve(irr_func, 0.1)[0]  # 初始猜测IRR为0.1，可根据实际情况调整

# 计算payback year
cumulative_cashflows = [initial_investment]  # 初始投资先作为第一个累积现金流
for i in range(1, len(net_inflows)):
    cumulative_cashflow = cumulative_cashflows[i - 1] + net_inflows[i]
    cumulative_cashflows.append(cumulative_cashflow)
    if cumulative_cashflow >= 0:
        payback_year = i + 1  # 加1是因为列表索引从0开始，而年份从1开始
        break

print("计算得到的IRR:", irr)
print("计算得到的Payback Year:", payback_year)

计算得到的IRR: 0.0902645003387917
计算得到的Payback Year: 11


IRR分布在8%左右，能得到不错的对比

#### 计算LCOE

In [461]:
import math

def calculate_lcoe(capital_cost, initial_opex, life, annual_energy_production, inflation_rate):

    total_cost_present_value = capital_cost
    opex_sum = 0
    annual_energy_sum = 0

    for year in range(1, life + 1):
        total_cost_present_value += initial_opex / (1 + inflation_rate) ** year
        opex_sum += initial_opex / (1 + inflation_rate) ** year
        annual_energy_sum += annual_energy_production / (1 + inflation_rate) ** year

    lcoe = total_cost_present_value / annual_energy_sum
    return lcoe

capital_cost = typical_unit.loc[5, 'Roof area'] * 1000
initial_opex = typical_unit.loc[5, 'Roof area'] * 10
life = 25
annual_energy_production = classified_SC1['PV'].sum()
inflation_rate = 0.065

lcoe = calculate_lcoe(capital_cost, initial_opex, life, annual_energy_production, inflation_rate)
print("LCOE:", lcoe, "CNY/kWh")

LCOE: 0.6834298538973576 CNY/kWh


分布式光伏度电成本低于当地工商业用户的电价。说明用户方在目前的技术和成本水平下已经能够接入平价。从发电端来看，分布式光伏度电成本均高于当地脱硫煤价。揭示出，在目前的技术水平下，分布式光伏在发电侧实现平价上网仍存在一定难度。

并且遮挡率和LCOE成正相关关系，表示了在同一个地区下，社区形态的复杂性带来的不同遮挡率对LCOE存在明显的影响，需要避开遮挡率大的社区。